In [2]:
# https://algotrading101.com/learn/yfinance-guide/

In [ ]:
from google.colab import files
import yfinance as yf
from google.cloud import bigquery
from google.colab import drive
import os
import datetime
import pytz
import numpy as np
from numpy.random import choice, uniform
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import lead, col, mean, stddev
from pyspark.sql.window import Window
from pyspark.ml.functions import vector_to_array
from pyspark.ml.feature import MinMaxScaler, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import SGD

# import elephas
# from elephas.ml_model import ElephasEstimator

import matplotlib.pyplot as plt

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
#generate credential key (json file) by creating a new project on Bigquery.
uploaded = files.upload()

Saving stock-prediction-439018-8ae1c7ee81a8.json to stock-prediction-439018-8ae1c7ee81a8 (1).json


In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/stock-prediction-439018-8ae1c7ee81a8.json'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# aapl_historical = aapl.history(start="2020-06-02", end="2020-06-07", interval="1m")
# data = yf.download("AMZN AAPL GOOG", start="2017-01-01", end="2017-04-30")
# data = yf.download("AMZN AAPL GOOG", start="2017-01-01", end="2017-04-30", group_by='tickers')
# apple = yf.Ticker("aapl")
# apple.info
# apple.actions
# apple.dividends
# apple.splits
# opt = aapl.option_chain(date='2020-07-24')
# opt.calls
# opt.puts
# opt.options
# apple.balance_sheet

In [ ]:
#PG, AAPL, GOOGL
symbols = ['GOOGL']
google = yf.Ticker('GOOGL')

In [1]:
stock_data = google.history(period = 'max')

In [ ]:
client = bigquery.Client(project='stock-prediction-439018')

In [ ]:
# create Dataset
dataset_id = 'stock-prediction-439018.stock'
dataset = bigquery.Dataset(dataset_id)
dataset = client.create_dataset(dataset, exists_ok=True)

stock


In [ ]:
# To create table in the existing dataset
table_id = 'stock-prediction-439018.stock.historical_data'
client.delete_table(table_id, not_found_ok=True)
job = client.load_table_from_dataframe(stock_data, table_id)
job.result()

LoadJob<project=stock-prediction-439018, location=US, id=d494eb4a-95fa-4af7-835d-2ec89288b843>

In [ ]:
# Sets up the configuration for a local Spark application named "stock-pred" using 2 CPU cores and Initializes a SparkContext
spark = SparkSession.builder.appName('stock-prediction').master("local[2]").getOrCreate()
# spark.stop()
# Initializes a SparkContext, which serves as the main interface for the Spark application/cluster
sc = spark.sparkContext

In [ ]:
client = bigquery.Client()

df = client.query('''select * from stock-prediction-439018.stock.historical_data;''').result().to_dataframe()
!rm '/content/drive/My Drive/Colab Notebooks/historical_data.csv'
df.to_csv('historical_data.csv')

!mv historical_data.csv '/content/drive/My Drive/Colab Notebooks/'

In [ ]:
df = spark.read.csv('/content/drive/My Drive/Colab Notebooks/historical_data.csv', header = True)[['Date','Open']]
df = df.orderBy('Date', ascending = True)
df = df.withColumn('Open', col('Open').cast('float'))

In [ ]:
for n in range(1,61):
  df = df.withColumn('stock_open_after_{}_days'.format(n), lead('Open',n).over(Window.orderBy('Date')))

df = df.withColumnsRenamed({'Open':'stock_open_after_0_days','stock_open_after_60_days':'target_price'})
df = df.na.drop()

In [ ]:
# import matplotlib.pyplot as plt

# hist_data = df.select('stock_open_after_20_days').rdd.map(lambda x: x[0]).histogram(10)

# bin_edges = hist_data[0]
# bin_counts = hist_data[1]

# # Plotting the histogram
# plt.bar(bin_edges[:-1], bin_counts, width=bin_edges[1] - bin_edges[0], align='edge')
# plt.xlabel('Feature Value')
# plt.ylabel('Frequency')
# plt.title('Histogram of Feature Distribution')
# plt.show()

In [ ]:
columns_to_scale = [col for col in df.columns if col.startswith("stock_open_after_") or col == "target_price"]

In [ ]:
# Scaling using Standardization technique

# scaling_stats = df.select(
#     *[F.mean(col).alias(f"{col}_mean") for col in columns_to_scale]+
#     [F.stddev(col).alias(f"{col}_stddev") for col in columns_to_scale]
# ).first()

# scaling_dict = {col: (scaling_stats[f"{col}_mean"], scaling_stats[f"{col}_stddev"]) for col in columns_to_scale}

# for col in columns_to_scale:
#     mean_value = scaling_dict[col][0]
#     stddev_value = scaling_dict[col][1]

#     df = df.withColumn(f"{col}_scaled",(F.col(col) - mean_value) / stddev_value)

# df_scaled = df.select([col for col in df.columns if col.endswith("scaled") or col == 'Date'])

In [ ]:
# Scaling using Min Max Scaling technique

global_min = df.select([F.min(col).alias(col) for col in columns_to_scale]).select(F.least(*columns_to_scale)).first()[0]
global_max = df.select([F.max(col).alias(col) for col in columns_to_scale]).select(F.greatest(*columns_to_scale)).first()[0]

print('Global Min:', global_min)
print('Global Max:', global_max)

scaled_columns = [((F.col(c) - global_min) / (global_max - global_min)).alias(f"{c}_scaled") for c in columns_to_scale]

df_scaled = df.select('Date', *scaled_columns)
df_scaled.show()

Global Min: 2.4761006832122803
Global Max: 190.05780029296875


In [ ]:
inputCols = ['stock_open_after_{}_days_scaled'.format(n) for n in range(60)]

assembler = VectorAssembler(inputCols = inputCols, outputCol = 'scaled_input_features')
df_assembled = assembler.transform(df_scaled)
df_assembled.show(4)

+--------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+----------------------------

In [ ]:
# df_expanded = df_assembled.withColumn("features_expanded", vector_to_array("scaled_input_features"))
# df_expanded.select("scaled_input_features","features_expanded").show(truncate=False)

In [ ]:
# Train Test split for time series data
eighty_perc_entry = int(df_assembled.count()*0.8)
split_date = df_assembled.limit(eighty_perc_entry).tail(1)[0].Date

train_df = df_assembled.filter(df_assembled['Date']< split_date).select(['scaled_input_features','target_price_scaled'])
test_df = df_assembled.filter(df_assembled['Date']>= split_date).select(['scaled_input_features','target_price_scaled'])

In [ ]:
train_pd = train_df.select('scaled_input_features', 'target_price_scaled').toPandas()

X_train = np.array(train_pd['scaled_input_features'].tolist())
y_train = np.array(train_pd['target_price_scaled'].tolist())

test_pd = test_df.select('scaled_input_features', 'target_price_scaled').toPandas()

X_test = np.array(test_pd['scaled_input_features'].tolist())
y_test = np.array(test_pd['target_price_scaled'].tolist())

In [ ]:
# def custom_loss(y_true, y_pred):
#     mae = tensorflow.keras.losses.mae(y_true, y_pred)
#     mse = tensorflow.keras.losses.mse(y_true, y_pred)
#     return mae + mse

In [3]:
# file_path = '/content/drive/My Drive/Colab Notebooks/'

# !rm file_path + 'X_train.npy'
# np.save(file_path + 'X_train.npy', X_train)

# !rm file_path + 'y_train.npy'
# np.save(file_path + 'y_train.npy', y_train)

# !rm file_path + 'X_test.npy'
# np.save(file_path + 'X_test.npy', X_test)

# !rm file_path + 'y_test.npy'
# np.save(file_path + 'y_test.npy', y_test)

In [4]:
# file_path = '/content/drive/My Drive/Colab Notebooks/'

# X_train = np.load(file_path + 'X_train.npy')
# y_train = np.load(file_path + 'y_train.npy')
# X_test = np.load(file_path + 'X_test.npy')
# y_test = np.load(file_path + 'y_test.npy')

In [ ]:
input_dim = len(X_train[0])

model = Sequential()
model.add(LSTM(units = 16, return_sequences = True, input_shape = (input_dim, 1), activity_regularizer = regularizers.L2(0.25)))
model.add(Dropout(0.35))
# model.add(LSTM(units = 64, return_sequences = True, activity_regularizer = regularizers.L2(0.25)))
# model.add(Dropout(0.4))
# model.add(LSTM(units = 64, return_sequences = True, activity_regularizer = regularizers.L2(0.25)))
# model.add(Dropout(0.3))
model.add(LSTM(units = 16, activity_regularizer = regularizers.L2(0.25)))
model.add(Dropout(0.3))
model.add(Dense(1))

adam_optimizer = Adam(learning_rate=0.001)
model.compile(loss= 'mean_squared_error', optimizer = adam_optimizer, metrics= ['mse'])
cb = [EarlyStopping(monitor = 'val_loss', min_delta = 0.0001, patience = 4, verbose=1)]
model.fit(X_train, y_train, epochs = 400, validation_split = 0.2, batch_size = 64, verbose = 2, callbacks = cb)

Epoch 1/400
51/51 - 11s - loss: 0.0266 - mse: 0.0043 - val_loss: 0.1449 - val_mse: 0.0142 - 11s/epoch - 207ms/step
Epoch 2/400
51/51 - 2s - loss: 0.0092 - mse: 7.3291e-04 - val_loss: 0.0917 - val_mse: 0.0024 - 2s/epoch - 44ms/step
Epoch 3/400
51/51 - 2s - loss: 0.0060 - mse: 4.4437e-04 - val_loss: 0.0623 - val_mse: 0.0016 - 2s/epoch - 45ms/step
Epoch 4/400
51/51 - 3s - loss: 0.0042 - mse: 3.2183e-04 - val_loss: 0.0459 - val_mse: 9.8969e-04 - 3s/epoch - 53ms/step
Epoch 5/400
51/51 - 5s - loss: 0.0031 - mse: 2.8147e-04 - val_loss: 0.0347 - val_mse: 5.4308e-04 - 5s/epoch - 92ms/step
Epoch 6/400
51/51 - 3s - loss: 0.0024 - mse: 2.3876e-04 - val_loss: 0.0282 - val_mse: 2.8835e-04 - 3s/epoch - 66ms/step
Epoch 7/400
51/51 - 2s - loss: 0.0019 - mse: 2.0597e-04 - val_loss: 0.0221 - val_mse: 4.5049e-04 - 2s/epoch - 44ms/step
Epoch 8/400
51/51 - 2s - loss: 0.0016 - mse: 2.1018e-04 - val_loss: 0.0181 - val_mse: 4.3483e-04 - 2s/epoch - 44ms/step
Epoch 9/400
51/51 - 2s - loss: 0.0013 - mse: 2.2540e-

In [ ]:
predictions = model.predict(X_test)
loss = mean_squared_error(y_test, predictions)
loss_percentage = (loss / y_test.mean()) * 100
print('mean_squared_error', loss_percentage)

loss = mean_absolute_error(y_test, predictions)
loss_percentage = (loss / y_test.mean()) * 100
print('mean_absolute_error', loss_percentage)

32/32 [==============================] - 2s 15ms/step
mean_squared_error 0.6333399498110378
mean_absolute_error 8.67392831430407


In [ ]:
predictions_train = model.predict(X_train)
loss = mean_squared_error(y_train, predictions_train)
loss_percentage = (loss / y_train.mean()) * 100
print('mean_squared_error', loss_percentage)

loss = mean_absolute_error(y_train, predictions_train)
loss_percentage = (loss / y_train.mean()) * 100
print('mean_absolute_error', loss_percentage)

126/126 [==============================] - 2s 16ms/step
mean_squared_error 0.18904507329011627
mean_absolute_error 8.364537396308565


In [ ]:
# sgd = optimizers.legacy.SGD(learning_rate=0.001, momentum= 0.9)

estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_json())
estimator.setFeaturesCol('scaled_input_features')
estimator.setLabelCol('target_price_scaled')
# estimator.set_optimizer_config(optimizers.serialize(sgd))
estimator.set_optimizer_config({'class_name': 'Adam', 'config': {'learning_rate': 0.001}})
estimator.set_mode("synchronous")
estimator.set_loss("mean_squared_error")
estimator.set_metrics(['mse'])
estimator.set_epochs(10)
estimator.set_batch_size(64)
estimator.set_validation_split(0.2)
estimator.set_categorical_labels(False)
estimator.set_verbosity(2)

ElephasEstimator_523f4b4bafd9

In [ ]:
pipeline = Pipeline(stages=[estimator])
fitted_pipeline = pipeline.fit(train_df)

>>> Fit model
>>> Synchronous training complete.


In [ ]:
# Evaluate Spark model by evaluating the underlying model
prediction = fitted_pipeline.transform(train_df)
pnl = prediction.select("target_price_scaled", "prediction")

In [ ]:
# prediction_and_label = pnl.rdd.map(lambda row: (row.label, row.prediction))
evaluator_r2 = RegressionEvaluator(labelCol="target_price_scaled", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(pnl)

evaluator_mae = RegressionEvaluator(labelCol="target_price_scaled", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(pnl)

evaluator_rmse = RegressionEvaluator(labelCol="target_price_scaled", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(pnl)

print(f"R2: {r2}")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")

R2: 0.9591101824903159
MAE: 0.015461859214017245
RMSE: 0.02105354682017697


In [ ]:
prediction_test = fitted_pipeline.transform(test_df)
pnl_test = prediction_test.select("target_price_scaled", "prediction")

In [ ]:
# prediction_and_label = pnl.rdd.map(lambda row: (row.label, row.prediction))
evaluator_r2 = RegressionEvaluator(labelCol="target_price_scaled", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(pnl_test)

evaluator_mae = RegressionEvaluator(labelCol="target_price_scaled", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(pnl_test)

evaluator_rmse = RegressionEvaluator(labelCol="target_price_scaled", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(pnl_test)

print(f"R2: {r2}")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")

R2: 0.8948028108421602
MAE: 0.036075425062921926
RMSE: 0.042920647742160235


In [ ]:
# def model(x_train, y_train, x_test, y_test):
#     input_dim = len(train_df.select('scaled_features').first()[0])

#     model = Sequential()
#     model.add(LSTM(units = {{choice([256, 512, 1024])}}, return_sequences = True, input_shape = (input_dim, 1), activity_regularizer = regularizers.l2(0.25)))
#     model.add(Dropout({{uniform(0, 0.6)}}))
#     # model.add(LSTM(units = {{choice([256, 512, 1024])}}, return_sequences = True, activity_regularizer = regularizers.L2(0.25)))
#     # model.add(Dropout(0.2))
#     # model.add(LSTM(units = {{choice([256, 512, 1024])}}, return_sequences = True, activity_regularizer = regularizers.L2(0.25)))
#     # model.add(Dropout(0.25))
#     model.add(LSTM(units = 64, activity_regularizer = regularizers.L2(0.25)))
#     model.add(Dropout({{uniform(0, 0.5)}}))
#     model.add(Dense(1))

#     model.compile(loss= 'mean_squared_error', optimizer = 'adam', metrics= ['mse'])
#     cb = [EarlyStopping(monitor = 'val_loss', min_delta = 0.0001, patience = 4, verbose=1)]
#     model.fit(X_train, y_train, epochs = 400, validation_split = 0.2, batch_size = {{choice([64, 128])}}, verbose = 2, callbacks = cb)

#     score, acc = model.evaluate(x_test, y_test, show_accuracy=True, verbose=0)
#     print('Test accuracy:', acc)
#     return {'loss': -acc, 'status': STATUS_OK, 'model': model.to_json()}

In [ ]:
# from elephas.hyperparam import HyperParamModel
# def data():
#   return X_train, y_train, X_test, y_test
# data = data()

# hyperparam_model = HyperParamModel(sc)
# hyperparam_model.minimize(model=model, data=data, max_evals=5)
# result = hyperparam_model.best_ensemble(nb_ensemble_models=10, model=model, data=data, max_evals=5)

(array([[1.07795793e-04, 2.42205306e-04, 1.53839726e-03, ...,
         1.29277953e-03, 8.32318257e-04, 6.25579181e-04],
        [2.42205306e-04, 1.53839726e-03, 1.60360399e-03, ...,
         8.32318257e-04, 6.25579181e-04, 2.78692849e-03],
        [1.53839726e-03, 1.60360399e-03, 7.67869152e-04, ...,
         6.25579181e-04, 2.78692849e-03, 2.14524037e-03],
        ...,
        [3.83245057e-01, 3.82104359e-01, 3.79474727e-01, ...,
         4.03053004e-01, 4.11113074e-01, 4.11099657e-01],
        [3.82104359e-01, 3.79474727e-01, 3.79296541e-01, ...,
         4.11113074e-01, 4.11099657e-01, 4.04107116e-01],
        [3.79474727e-01, 3.79296541e-01, 3.88041754e-01, ...,
         4.11099657e-01, 4.04107116e-01, 3.93667908e-01]]),
 array([ 4.62393522,  4.50461197,  4.43097019, ..., 79.25026703,
        77.3090744 , 75.66363525]))